# Introduction to the Python control system

This is the introductory example explaining the connection setup between Python and the **Term** Terminal Software running the **Thales** Workstation Software.

In [1]:
import sys
from thales_remote.connection import ThalesRemoteConnection
from thales_remote.script_wrapper import PotentiostatMode,ThalesRemoteScriptWrapper
import math
import cmath

# Utility functions

First, two utility functions are declared.  
With the first function the complex impedance is output to the console. The second function measures an impedance spectrum from individual impedance measuring points.

In [2]:
def printImpedance(impedance):
    print("Impedance: " + str(abs(impedance)) + " ohm, " + str(cmath.phase(impedance)) + " rad")
    return

def spectrum(scriptHandle, lower_frequency, upper_frequency, number_of_points):
    log_lower_frequency = math.log(lower_frequency)
    log_upper_frequency = math.log(upper_frequency)
    log_interval_spacing = (log_upper_frequency - log_lower_frequency) / (number_of_points - 1)
    
    for i in range(number_of_points):
        current_frequency = math.exp(log_lower_frequency + log_interval_spacing * i)
        print("Frequency: " + str(current_frequency))
        printImpedance(scriptHandle.getImpedance(current_frequency))
        
    return

# Connect to the Term software

Before calling the following commands, the Term software must be started and the Thales start screen must be displayed, in which methods can be selected.

In this case the Term runs on the same computer as Python, so **"localhost"** is chosen here as IP address, but you could also specify the IP address of the computer in the network, on which the Term software is running and to which the workstation is connected via USB.

In [3]:
TARGET_HOST = "localhost"

if __name__ == "__main__":
    zenniumConnection = ThalesRemoteConnection()
    connectionSuccessful = zenniumConnection.connectToTerm(TARGET_HOST, "ScriptRemote")
    if connectionSuccessful:
        print("connection successfull")
    else:
        print("connection not possible")
        sys.exit()

connection successfull


# Initialize the application and start the remote software in Thales

If the connection to the term is successfully established, the [ThalesRemoteConnection](https://doc.zahner.de/thales_remote/connection.html#thales_remote.connection.ThalesRemoteConnection) object that manages the connection to the Thales software. This object is passed to the constructor of a new [ThalesRemoteScriptWrapper](https://doc.zahner.de/thales_remote/script_wrapper.html#thales_remote.script_wrapper.ThalesRemoteScriptWrapper) object.

The [ThalesRemoteScriptWrapper](https://doc.zahner.de/thales_remote/script_wrapper.html#thales_remote.script_wrapper.ThalesRemoteScriptWrapper) class implements the commands as methods, which are available via [Remote2](https://doc.zahner.de/Remote.pdf).

In [ ]:
    zahnerZennium = ThalesRemoteScriptWrapper(zenniumConnection)
    zahnerZennium.forceThalesIntoRemoteScript()

# Offset calibration

The first step is to perform an offset calibration. The instrument performs automatic offset calibrations but it is recommended to calibrate the offsets manually after the instrument has warmed up for half an hour.

In [ ]:
    zahnerZennium.calibrateOffsets()

# DC measurement

## Potentiostatic measurement

A voltage of 1.0 V is output potentiostatically as an example. For this, potentiostatic is set first. Then the potential is set and the potentiostat is switched on.

In [ ]:
    zahnerZennium.setPotentiostatMode(PotentiostatMode.POTMODE_POTENTIOSTATIC)
    zahnerZennium.setPotential(1.0)
    zahnerZennium.enablePotentiostat()

After the potentiostat is switched on, voltage and current are measured several times in a loop.

In [7]:
    for i in range(5):
        print(f"Potential:\t{zahnerZennium.getPotential()}\tV")
        print(f"Current:\t{zahnerZennium.getCurrent()}\tA")

Potential:	0.9998795	V
Current:	9.974371e-09	A
Potential:	0.9998603	V
Current:	9.985522e-09	A
Potential:	0.9998633	V
Current:	9.986938e-09	A
Potential:	0.9998643	V
Current:	9.987265e-09	A
Potential:	0.9998643	V
Current:	9.987638e-09	A


## Galvanostatic measurement

After the potentiostatic measurement, galvanostatic measurement is performed with 20 nA.

In [8]:
    zahnerZennium.disablePotentiostat()
    zahnerZennium.setPotentiostatMode(PotentiostatMode.POTMODE_GALVANOSTATIC)
    zahnerZennium.setCurrent(20e-9)
    zahnerZennium.enablePotentiostat()

    for i in range(5):
        print(f"Potential:\t{zahnerZennium.getPotential()}\tV")
        print(f"Current:\t{zahnerZennium.getCurrent()}\tA")

Potential:	1.94944	V
Current:	1.992362e-08	A
Potential:	1.993915	V
Current:	1.992354e-08	A
Potential:	1.994116	V
Current:	1.992357e-08	A
Potential:	1.994305	V
Current:	1.992382e-08	A
Potential:	1.994422	V
Current:	1.992356e-08	A


# Impedance measurement

## Parameterization of the impedance measurement

For the impedance measurement, the measuring frequency, the measuring amplitude and the number of periods to be averaged are now set.

In [ ]:
    zahnerZennium.disablePotentiostat()
    zahnerZennium.setPotentiostatMode(PotentiostatMode.POTMODE_POTENTIOSTATIC)
    zahnerZennium.setPotential(1.0)
    zahnerZennium.enablePotentiostat()
    zahnerZennium.setFrequency(2000)
    zahnerZennium.setAmplitude(10e-3)
    zahnerZennium.setNumberOfPeriods(3)

## Execute the measurement

Since the potentiostat is still switched on from the DC measurement, the impedance spectrum is now measured at the set DC potential of 1 V.

<div class="alert alert-block alert-info">
<b>Note:</b> If the potentiostat is set to potentiostatic before the impedance measurement and is switched off, the measurement is performed at the open circuit voltage/potential.
</div>
    

In [10]:
    printImpedance(zahnerZennium.getImpedance())
    printImpedance(zahnerZennium.getImpedance(2000))
    printImpedance(zahnerZennium.getImpedance(2000, 10e-3, 3))

    spectrum(zahnerZennium, 1000, 2e5, 10)

Impedance: 548664.9346194816 ohm, -1.5554111026239594 rad
Impedance: 548173.1660123469 ohm, -1.5544576968941222 rad
Impedance: 548367.4877388338 ohm, -1.5551072986627177 rad
Frequency: 999.9999999999998
Impedance: 1092168.7781657192 ohm, -1.5532157206942645 rad
Frequency: 1801.64823065441
Impedance: 606590.1924693475 ohm, -1.5535515401416211 rad
Frequency: 3245.936347020167
Impedance: 337333.38778721564 ohm, -1.5567266949188723 rad
Frequency: 5848.035476425735
Impedance: 188612.52947776296 ohm, -1.5592697959397968 rad
Frequency: 10536.102768906649
Impedance: 104801.92255412112 ohm, -1.5647391489656668 rad
Frequency: 18982.350911593698
Impedance: 58230.39139701879 ohm, -1.5671298482906713 rad
Frequency: 34199.51893353398
Impedance: 32190.079340705266 ohm, -1.5730165778841319 rad
Frequency: 61615.50277583351
Impedance: 17900.02994152803 ohm, -1.5689672785552131 rad
Frequency: 111009.46155696237
Impedance: 9790.83542962499 ohm, -1.5577327186024725 rad
Frequency: 200000.00000000006
Impedan

## Switch off potentiostat

After the measurement, the potentiostat is switched off and **the amplitude must be set to 0** again after the impedance measurement.

In [ ]:
    zahnerZennium.disablePotentiostat()
    zahnerZennium.setAmplitude(0)

# Close the connection

In [12]:
    zenniumConnection.disconnectFromTerm()
    print("finish")

finish
